# 앙상블 학습(Ensemble Learning)

앙상블 학습은 여러 개의 모델을 결합하여 더 나은 성능을 얻는 기법임. 개별 모델의 예측을 결합해 오차를 줄이고 일반화 성능을 향상시킴.

주요 방식:
- **배깅(Bagging)**: 여러 모델을 병렬로 학습시키고 예측을 평균 또는 투표로 결합함. 대표적으로 **랜덤 포레스트(Random Forest)** 가 있음.
- **부스팅(Boosting)**: 모델을 순차적으로 학습시키며 이전 모델이 잘못 예측한 데이터에 가중치를 부여함. 대표적으로 **AdaBoost**, **Gradient Boosting**이 있음.
- **스태킹(Stacking)**: 서로 다른 모델들의 예측 결과를 메타 모델이 결합하여 최종 예측을 함.

앙상블 기법은 정형 데이터(structured data)개별 모델의 한계를 보완하여 성능을 높이는 데 유용함.

### 랜덤 포레스트(Random Forest)

랜덤 포레스트는 **배깅(Bagging)** 기반의 앙상블 학습 기법으로, 여러 개의 **결정 트리(Decision Tree)** 를 결합하여 예측 성능을 높임. 

작동 방식:
1. **부트스트랩 샘플링**: 원본 데이터에서 여러 샘플을 **부트스트랩 샘플** 방식으로 무작위 추출함. 이는 데이터 셋에서 중복을 허용하여 일부 데이터를 여러 번 사용하거나 아예 사용하지 않을 수 있게 하는 방법으로, 각 트리마다 다른 학습 데이터 세트를 구성하여 모델의 다양성을 높임.
2. **특성 무작위 선택**: 각 트리의 분기 시, 일부 랜덤하게 선택된 특성들 중에서 분할을 수행하여 트리 간의 상관성을 낮추고 다양성을 높임.
3. **예측 결합**: 분류 문제에서는 다수결 투표로, 회귀 문제에서는 평균을 내어 최종 예측을 만듦.

부트스트랩 샘플링을 통해 데이터 변동에 강하고, 과적합을 줄이며 높은 예측 성능을 제공함.

In [2]:
import numpy as np  # 배열 및 수치 연산을 위한 라이브러리
import pandas as pd  # 데이터 분석을 위한 라이브러리
from sklearn.model_selection import train_test_split  # 데이터셋을 학습용과 테스트용으로 나누는 함수

# 와인 데이터 불러옴. 데이터는 CSV 파일로 제공되며 URL을 통해 직접 다운로드
wine = pd.read_csv('https://bit.ly/wine_csv_data')

# 독립 변수 선택: 'alcohol', 'sugar', 'pH' 컬럼만 사용해 모델을 훈련
# DataFrame에서 원하는 컬럼을 선택한 후, 이를 넘파이 배열로 변환
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()

# 종속 변수 선택: 'class' 컬럼을 사용해 와인의 클래스를 예측
# DataFrame의 'class' 컬럼을 넘파이 배열로 변환
target = wine['class'].to_numpy()

# 데이터를 학습용과 테스트용으로 나눔
# test_size=0.2는 전체 데이터의 20%를 테스트 세트로, 나머지 80%를 학습 세트로 사용
# random_state=42는 결과의 재현성을 위해 난수 시드를 고정함
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [3]:
from sklearn.model_selection import cross_validate  # 모델의 교차 검증을 위한 함수
from sklearn.ensemble import RandomForestClassifier  # 랜덤 포레스트 분류 모델

# 랜덤 포레스트 분류 모델 생성
# n_jobs=-1로 설정해 모든 CPU 코어를 사용해 병렬 처리
# random_state=42로 설정해 결과의 재현성을 보장
rf = RandomForestClassifier(n_jobs=-1, random_state=42)

# 교차 검증 수행
# 학습 세트(train_input, train_target)를 사용해 교차 검증을 수행하고, 각 훈련 및 검증 단계에서의 성능을 수집
# return_train_score=True로 설정해 학습 단계에서의 점수도 반환
# n_jobs=-1로 설정해 교차 검증을 병렬 처리
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)

# 학습 점수와 테스트 점수의 평균 출력
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8903229806766861


In [4]:
# 랜덤 포레스트 모델을 학습 데이터로 훈련
rf.fit(train_input, train_target)

# 특성 중요도 출력
# 모델이 각 특성('alcohol', 'sugar', 'pH')에 대해 예측에 얼마나 기여했는지를 나타내는 중요도 값을 반환
print(rf.feature_importances_)

[0.23183515 0.50059756 0.26756729]


In [5]:
# 랜덤 포레스트 분류 모델 생성
# oob_score=True로 설정해 OOB (Out-of-Bag) 샘플을 사용한 평가 점수를 계산
# n_jobs=-1로 모든 CPU 코어를 사용해 병렬 처리, random_state=42로 결과의 재현성 보장
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)

# 학습 데이터로 모델 훈련
rf.fit(train_input, train_target)

# OOB 샘플을 사용해 계산한 점수 출력
print(rf.oob_score_)

0.8945545507023283


# 엑스트라 트리(Extra Trees, Extremely Randomized Trees)

엑스트라 트리는 **랜덤 포레스트** 와 유사한 앙상블 학습 기법으로, 다수의 **결정 트리(Decision Tree)** 를 결합하여 예측 성능을 높임. 그러나 랜덤 포레스트와 달리, 분할 시 무작위성을 더욱 강화하여 모델의 변동성을 낮춤.

작동 방식:
1. **전체 데이터 사용**: 랜덤 포레스트와 달리, 부트스트랩 샘플링을 사용하지 않고 원본 데이터 전체를 사용하여 각 트리를 학습시킴.
2. **특성 무작위 선택과 무작위 분할**: 각 트리의 분기 시 일부 랜덤하게 선택된 특성 중, 최적의 분할 값 대신 임의의 분할 값을 사용하여 트리의 분할을 수행함. 이로써 모델의 무작위성을 높이고, 트리 간의 상관성을 낮춤.
3. **예측 결합**: 랜덤 포레스트와 마찬가지로, 분류 문제에서는 다수결 투표, 회귀 문제에서는 평균을 내어 최종 예측을 만듦.

엑스트라 트리는 계산 속도가 빠르고, 데이터 변동성에 강하며, 과적합을 줄이는 효과가 있음.

In [6]:
from sklearn.ensemble import ExtraTreesClassifier  # 엑스트라 트리 분류 모델

# 엑스트라 트리 분류 모델 생성
# n_jobs=-1로 모든 CPU 코어를 사용해 병렬 처리
# random_state=42로 결과의 재현성 보장
et = ExtraTreesClassifier(n_jobs=-1, random_state=42)

# 교차 검증 수행
# 학습 세트(train_input, train_target)를 사용해 교차 검증을 수행하고, 각 훈련 및 검증 단계에서의 성능을 수집
# return_train_score=True로 설정해 학습 단계의 점수도 반환
# n_jobs=-1로 설정해 교차 검증을 병렬 처리
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)

# 학습 점수와 테스트 점수의 평균 출력
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


In [7]:
# 엑스트라 트리 모델을 학습 데이터로 훈련
et.fit(train_input, train_target)

# 특성 중요도 출력
# 모델이 각 특성('alcohol', 'sugar', 'pH')에 대해 예측에 얼마나 기여했는지를 나타내는 중요도 값을 반환
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


# 그레이디언트 부스팅(Gradient Boosting)

그레이디언트 부스팅은 **부스팅(Boosting)** 기반의 앙상블 학습 기법으로, 여러 약한 학습기(주로 결정 트리)를 순차적으로 학습시켜 강력한 예측 모델을 만드는 방식임.

작동 방식:
1. **순차적 학습**: 모델을 순차적으로 학습하며, 이전 모델의 오차(잔차)를 다음 모델이 보완하도록 함.
2. **잔차 최소화**: 각 단계에서 이전 모델의 예측 오차를 줄이기 위해, 손실 함수의 **그레이디언트(Gradient)** 를 계산하여 이를 보정할 수 있는 새로운 모델을 학습시킴.
3. **모델 결합**: 최종 예측은 모든 모델의 예측을 가중합으로 결합하여 계산함.

특징:
- **고성능**: 이전 단계의 오차를 보정하며 학습하기 때문에 복잡한 데이터에 대해 높은 예측 성능을 가짐.
- **하이퍼파라미터 중요성**: 학습률(Learning Rate)이나 트리의 깊이 등의 하이퍼파라미터가 모델 성능에 큰 영향을 미치므로, 적절한 튜닝이 필요함.
- 대표적으로 **XGBoost**, **LightGBM**, **CatBoost**와 같은 변형 모델이 있음.

그레이디언트 부스팅은 다양한 문제에 강력한 성능을 제공하지만, 계산 비용이 크므로 주의가 필요함.

In [8]:
from sklearn.ensemble import GradientBoostingClassifier  # 그레이디언트 부스팅 분류 모델

# 그레이디언트 부스팅 분류 모델 생성
# random_state=42로 결과의 재현성 보장
gb = GradientBoostingClassifier(random_state=42)

# 교차 검증 수행
# 학습 세트(train_input, train_target)를 사용해 교차 검증을 수행하고, 각 훈련 및 검증 단계에서의 성능을 수집
# return_train_score=True로 설정해 학습 단계에서의 점수도 반환
# n_jobs=-1로 설정해 교차 검증을 병렬 처리
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

# 학습 점수와 테스트 점수의 평균 출력
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


In [9]:
# 그레이디언트 부스팅 분류 모델 생성
# n_estimators=500: 사용할 트리의 개수를 500개로 설정하여 더 많은 학습을 수행
# learning_rate=0.2: 학습률을 0.2로 설정하여 각 단계의 기여를 조정
# random_state=42: 결과의 재현성을 보장
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42)

# 교차 검증 수행
# 학습 세트(train_input, train_target)를 사용해 교차 검증을 수행하고, 각 훈련 및 검증 단계에서의 성능을 수집
# return_train_score=True로 설정해 학습 단계의 점수도 반환
# n_jobs=-1로 설정해 교차 검증을 병렬 처리
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

# 학습 점수와 테스트 점수의 평균 출력
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


In [10]:
# 그레이디언트 부스팅 모델을 학습 데이터로 훈련
gb.fit(train_input, train_target)

# 특성 중요도 출력
# 모델이 각 특성('alcohol', 'sugar', 'pH')에 대해 예측에 얼마나 기여했는지를 나타내는 중요도 값을 반환
print(gb.feature_importances_)

[0.15881044 0.67988912 0.16130044]


### 히스토그램 기반 그레이디언트 부스팅(Histogram-based Gradient Boosting)

히스토그램 기반 그레이디언트 부스팅은 **그레이디언트 부스팅** 의 한 변형으로, 학습 속도를 높이기 위해 **히스토그램 분할 방식** 을 사용하여 연속형 특성을 구간으로 나누어 처리함. 대용량 데이터에 특히 효율적임.

작동 방식:
1. **히스토그램 분할**: 연속형 특성을 일정 구간으로 나누어 **히스토그램(bin)** 을 생성함. 이를 통해 데이터의 분포를 빠르게 파악하고, 분할 기준을 효율적으로 선택할 수 있음.
2. **그레이디언트 기반 분할**: 각 히스토그램 구간 내에서 그레이디언트를 계산하여, 최적의 분할점을 선택하고 오차를 줄임.
3. **모델 결합**: 일반적인 그레이디언트 부스팅과 마찬가지로, 모든 모델의 예측을 가중합하여 최종 예측을 만듦.

특징:
- **속도 향상**: 히스토그램을 사용하여 계산량을 줄이므로, 대용량 데이터에서 학습 속도가 빠름.
- **메모리 절약**: 데이터 전체가 아닌 구간별 빈(bin)만을 저장하므로, 메모리 사용량이 줄어듦.
- **고성능**: XGBoost, LightGBM, CatBoost 등 최신 그레이디언트 부스팅 모델에서 활용되며, 특히 LightGBM은 이 방식을 최적화하여 효율성과 성능을 모두 극대화함.

히스토그램 기반 그레이디언트 부스팅은 대규모 데이터셋에서의 효율성과 정확성을 동시에 제공함.

In [11]:
from sklearn.experimental import enable_hist_gradient_boosting  # 히스토그램 기반 그레이디언트 부스팅을 활성화
from sklearn.ensemble import HistGradientBoostingClassifier  # 히스토그램 기반 그레이디언트 부스팅 분류 모델

# 히스토그램 기반 그레이디언트 부스팅 모델 생성
# random_state=42로 설정해 결과의 재현성을 보장
hgb = HistGradientBoostingClassifier(random_state=42)

# 교차 검증 수행
# 학습 세트(train_input, train_target)를 사용해 교차 검증을 수행하고, 각 훈련 및 검증 단계에서의 성능을 수집
# return_train_score=True로 설정해 학습 단계에서의 점수도 반환
scores = cross_validate(hgb, train_input, train_target, return_train_score=True)

# 학습 점수와 테스트 점수의 평균 출력
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

/Users/leesung-joon/jupyter_env/lib/python3.13/site-packages/sklearn/experimental/enable_hist_gradient_boosting.py:16: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  warnings.warn(


0.9321723946453317 0.8801241948619236


In [12]:
from sklearn.inspection import permutation_importance  # 특성 중요도를 평가하기 위한 함수

# 히스토그램 기반 그레이디언트 부스팅 모델을 학습 데이터로 훈련
hgb.fit(train_input, train_target)

# 순열 중요도 계산
# n_repeats=10: 각 특성에 대해 10번의 반복을 통해 중요도를 평가
# random_state=42: 결과의 재현성을 보장
# n_jobs=-1: 가능한 모든 CPU 코어를 사용하여 병렬 처리
result = permutation_importance(hgb, train_input, train_target, n_repeats=10, random_state=42, n_jobs=-1)

# 각 특성의 중요도 평균값 출력
# importances_mean은 각 특성에 대한 중요도의 평균값을 반환
print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]


In [13]:
# 테스트 데이터셋을 사용하여 순열 중요도 계산
# test_input과 test_target을 사용해 모델 성능에 대한 특성 중요도를 평가
# n_repeats=10: 각 특성에 대해 10번의 반복을 통해 중요도를 평가
# random_state=42: 결과의 재현성을 보장
# n_jobs=-1: 가능한 모든 CPU 코어를 사용하여 병렬 처리
result = permutation_importance(hgb, test_input, test_target, n_repeats=10, random_state=42, n_jobs=-1)

# 각 특성의 중요도 평균값 출력
# importances_mean은 각 특성에 대한 중요도의 평균값을 반환
print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


In [14]:
# 테스트 데이터셋을 사용하여 모델의 성능 평가
# hgb 모델에 대해 test_input(특성)과 test_target(타겟) 데이터를 사용하여 정확도(score)를 계산
score = hgb.score(test_input, test_target)

# 계산된 정확도 출력
print(score)

0.8723076923076923

# XGBoost (Extreme Gradient Boosting)

XGBoost는 **그레이디언트 부스팅(Gradient Boosting)** 을 기반으로 한 고성능 머신러닝 라이브러리로, 속도와 효율성을 극대화한 다양한 최적화 기법이 포함되어 있음. 주로 대용량 데이터와 복잡한 예측 문제에서 높은 성능을 발휘함.

작동 방식:
1. **그레이디언트 부스팅 기반 학습**: 기본적으로 그레이디언트 부스팅의 잔차를 최소화하는 방식으로 학습하며, 각 단계에서 이전 모델의 오차를 보정하는 새로운 모델을 추가함.
2. **정규화 및 가지치기**: 모델이 과적합되지 않도록 L1, L2 정규화와 함께, 불필요한 분기를 제거하는 **가지치기(pruning)**  기법을 사용함.
3. **병렬 처리**: 데이터의 여러 특성에 대해 병렬 처리가 가능하도록 하여 학습 속도를 크게 향상시킴.
4. **히스토그램 기반 분할**: 연속형 특성을 구간으로 나눠 히스토그램을 생성해 분할점을 선택하므로, 메모리와 계산 비용을 절약함.

특징:
- **고성능**: 정규화와 가지치기로 모델의 일반화 성능을 높이고, 빠른 학습과 예측을 지원함.
- **유연한 파라미터 조정**: 학습률(Learning Rate), 트리 깊이, 정규화 계수 등 다양한 하이퍼파라미터를 통해 모델을 세밀하게 튜닝할 수 있음.
- **강력한 확장성**: 대용량 데이터와 고차원 특성을 가진 데이터에 대해서도 뛰어난 성능을 보임.

XGBoost는 데이터 과학 경진 대회에서 자주 사용되는 모델로, 높은 예측 성능과 효율성 덕분에 널리 활용되고 있음.

In [16]:
from xgboost import XGBClassifier  # XGBoost 분류 모델 임포트

# XGBoost 모델 생성: 'tree_method'를 'hist'로 설정하여 히스토그램 기반 방식 사용
# random_state=42로 설정해 결과의 재현성을 보장
xgb = XGBClassifier(tree_method='hist', random_state=42)

# 교차 검증 수행
# 학습 세트(train_input, train_target)를 사용해 교차 검증을 수행하고, 각 훈련 및 검증 단계에서의 성능을 수집
# return_train_score=True로 설정해 학습 단계에서의 점수도 반환
scores = cross_validate(xgb, train_input, train_target, return_train_score=True)

# 학습 점수와 테스트 점수의 평균 출력
# train_score: 훈련 데이터에 대한 성능, test_score: 검증 데이터에 대한 성능
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9558403027491312 0.8782000074035686


In [28]:
from lightgbm import LGBMClassifier  # LightGBM 분류 모델 임포트

# LightGBM 모델 생성: random_state=42로 설정해 결과의 재현성을 보장
lgb = LGBMClassifier(random_state=42)

# 교차 검증 수행
# 학습 세트(train_input, train_target)를 사용해 교차 검증을 수행하고, 각 훈련 및 검증 단계에서의 성능을 수집
# return_train_score=True로 설정해 학습 단계에서의 점수도 반환
# n_jobs=-1로 설정해 가능한 모든 CPU 코어를 사용해 병렬 처리
scores = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)

[LightGBM] [Info] Number of positive: 3151, number of negative: 1006
[LightGBM] [Info] Number of positive: 3151, number of negative: 1007
[LightGBM] [Info] Number of positive: 3151, number of negative: 1006
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000405 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 373
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000598 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000778 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 370
[LightGBM] [Info] Total Bins 372
[LightGBM] [Info] Number of data points in the train set: 4158, number of used features: 3
[LightGBM] [Info] Number of positive: 3152, number of